# HW3 - Q3 [35 pts]

## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: <strong>REMOVE</strong> any print statements added to cells with "#export" that are used for debugging purposes befrore submitting because they will crash the autograder in Gradescope. Any additional cells can be used for testing purposes at the bottom. 
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove any comment that says "#export" because that will crash the autograder in Gradescope. We use this comment to export your code in these cells for grading.
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> import any additional libraries into this workbook.
</div>

All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  

<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`long_trips()`

`manhattan_trips()`

`weighted_profit()`

`final_output()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from the NYC-TLC dataset during auto-grading.

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove or modify the following utility functions:
</div>

`load_data()`

`main()`

<div class="alert alert-block alert-info">
    Do <strong>not</strong> change the below cell. Run it to initialize your PySpark instance. If you don't get any output, make sure your Notebook's Kernel is set to "PySpark" in the top right corner.
</div>

In [38]:
sc

Calculation started (calculation_id=b0c9559e-a5f5-a6ec-462e-91732c4b3a2d) in (session=4ec9558a-0044-0397-9059-1746c708fe75). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
<SparkContext master=athena appName=default>



<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remodify the below cell. It contains the function for loading data and all imports, and the function for running your code.
</div>

In [39]:
#export
from pyspark.sql.functions import *
from pyspark.sql import *

Calculation started (calculation_id=3ec9559e-ab22-9d09-76de-9049a9afe7c3) in (session=4ec9558a-0044-0397-9059-1746c708fe75). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [40]:
#### DO NOT CHANGE ANYTHING IN THIS CELL ####

def load_data(size='small'):
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'
    
    if size != 'small' and size != 'large':
        print("Invalid size parameter provided. Use only 'small' or 'large'.")
        return
    
    input_bucket = "s3://cse6242-hw3-q3"
    
    # Load Trip Data
    trip_path = '/'+size+'/yellow_tripdata*'
    trips = spark.read.csv(input_bucket + trip_path, header=True, inferSchema=True)
    print("Trip Count: ",trips.count()) # Prints # of trips (# of records, as each record is one trip)
    
    # Load Lookup Data
    lookup_path = '/'+size+'/taxi*'
    lookup = spark.read.csv(input_bucket + lookup_path, header=True, inferSchema=True)
    
    return trips, lookup

def main(size, bucket):
    # Runs your functions implemented above.
    
    print(user())
    trips, lookup = load_data(size=size)
    trips = long_trips(trips)
    mtrips = manhattan_trips(trips, lookup)
    wp = weighted_profit(trips, mtrips)
    final = final_output(wp, lookup)
    
    # Outputs the results for you to visually see
    final.show()
    
    # Writes out as a CSV to your bucket.
    final.write.csv(bucket)

Calculation started (calculation_id=e8c9559e-ac5c-f8c6-5202-409a4172256a) in (session=4ec9558a-0044-0397-9059-1746c708fe75). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


# Implement the below functions for this assignment:
<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> change any function inputs or outputs, and ensure that the dataframes your code returns align with the schema definitions commented in each function. Do <strong>NOT</strong> remove the #export comment from each of the code blocks either. This can prevent your code from being converted to a python file.
</div>

## 3.1 [1 pt] Update the `user()` function
This function should return your GT username, eg: gburdell3

In [41]:
#export
def user():
    # Returns a string consisting of your GT username.
    return 'hpavlovich3'

Calculation started (calculation_id=d6c9559e-adb3-6f27-6500-b0dbe2252771) in (session=4ec9558a-0044-0397-9059-1746c708fe75). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3.2 [2 pts] Update the `long_trips()` function
This function filters trips to keep only trips greater than or equal to 2 miles.

In [42]:
#export
def long_trips(trips):
    # Returns a Dataframe (trips) with Schema the same as :trips:
    df = trips.filter(trips.trip_distance >=2)
    return df

Calculation started (calculation_id=a2c9559e-aeec-10cd-0b68-38f7b67f2304) in (session=4ec9558a-0044-0397-9059-1746c708fe75). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3.3 [6 pts] Update the `manhattan_trips()` function

This function determines the top 20 locations with a `DOLocationID` in manhattan by passenger_count (pcount).

Example output formatting:

```
+--------------+--------+
| DOLocationID | pcount |
+--------------+--------+
|             5|      15|
|            16|      12| 
+--------------+--------+
```

In [43]:
#export
def manhattan_trips(trips, lookup):
    # Returns a Dataframe (mtrips) with Schema: DOLocationID, pcount
    lookup_hold = lookup.withColumnRenamed("LocationID", "DOLocationID")

    mtrips = trips.join(lookup_hold, ["DOLocationID"], "outer") \
            .filter(col("Borough") == "Manhattan")

    mtrips = groupBy("DOLocationID") \
        .agg(sum("passenger_count").alias("pcount")) \
        .select("DOLocationID", "pcount") \
        .orderBy(desc("pcount")) \
        .limit(20)
    
    return mtrips

Calculation started (calculation_id=16c9559e-b00c-a534-ccca-8544b036e583) in (session=4ec9558a-0044-0397-9059-1746c708fe75). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3.4 [6 pts] Update the `weighted_profit()` function
This function should determine the average `total_amount`, the total count of trips, and the total count of trips ending in the top 20 destinations and return the `weighted_profit` as discussed in the homework document.

Example output formatting:
```
+--------------+-------------------+
| PULocationID |  weighted_profit  |
+--------------+-------------------+
|            18| 33.784444421924436| 
|            12| 21.124577637149223| 
+--------------+-------------------+
```

In [44]:
#export
def weighted_profit(trips, mtrips): 
    # Returns a Dataframe (wp) with Schema: PULocationID, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    
    # average costs and total counts
    trip_hold = trips.agg(avg("total_amount").alias("avg_amount"),
                         count("*").alias("total_trips"))
    
    # top 20 destinations
    count_20 = trips.join(mtrips, ["DOLocationId"], "inner").count()
    
    # weighted_profit - count all trips and put it across all
    wp_hold = trips.groupBy("PULocationID") \
        .agg(count("*").alias("PU_count"), avg("total_amount").alias("PU_avg_count")) \
        .crossJoin(trip_hold)
    
    # add the counts in its own column
    wp = wp_hold.crossJoin(lit(count_20).alias("count_20")) \
        .withColumn("weighted_profit", ((col("PU_count") * col("count_20") * col("PU_avg_count") /
                    (col("total_trips")**2))).cast(DecimalType(24,10))
        .select("PULocationID", "weighted_profit")
    
    return(wp)

Calculation started (calculation_id=c6c9559e-b137-46ce-0118-41d7c83388b4) in (session=4ec9558a-0044-0397-9059-1746c708fe75). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation c6c9559e-b137-46ce-0118-41d7c83388b4 failed


  File "<stdin>", line 24
    return(wp)
    ^
SyntaxError: invalid syntax



## 3.5 [5 pts] Update the `final_output()` function
This function will take the results of `weighted_profit`, links it to the `borough` and `zone` and returns the top 20 locations with the highest `weighted_profit`.

Example output formatting:
```
+------------+---------+-------------------+
|    Zone    | Borough |  weighted_profit  |
+----------------------+-------------------+
| JFK Airport|   Queens|  16.95897820117925|
|     Jamaica|   Queens| 14.879835188762488|
+------------+---------+-------------------+
```

In [45]:
#export
def final_output(wp, lookup): 
    # Returns a Dataframe (final) with Schema: Zone, Borough, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    
    wp_1 = wp.withColumnRenamed("PULocationID", "LocationID") \
        .join(lookup, "LocationID", "inner") \
        .select("Zone", "Borough", "weighted_profit") \
        .orderBy(desc("weighted_profit")) \
        .limit(20)
    
    return wp_1

Calculation started (calculation_id=20c9559e-b25d-8dec-4577-f8a2362f2ca2) in (session=4ec9558a-0044-0397-9059-1746c708fe75). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


#### Testing

<div class="alert alert-block alert-info">
    You may use the below cell for any additional testing you need to do, however any code implemented below will not be run or used when grading.
</div>

In [46]:
main("large", "s3://cse6242-hpavlovich3/test0")

Calculation started (calculation_id=f2c9559e-b37b-9037-d4d0-c173871dacef) in (session=4ec9558a-0044-0397-9059-1746c708fe75). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation f2c9559e-b37b-9037-d4d0-c173871dacef failed
hpavlovich3
Trip Count:  187203269

  File "<stdin>", line 1, in <module>
  File "<stdin>", line 28, in main
  File "<stdin>", line 8, in manhattan_trips
NameError: name 'groupBy' is not defined

